In [1]:
import os
os.chdir('..')

In [2]:
import torch
from tqdm import tqdm
from src.pharmaconet.modeling import ModelingModule

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_PATH = 'model_weights/model.tar'
split_file = torch.load('dataset/pocket_to_ligands.pt')

In [4]:
score_threshold =  {
    "PiStacking_P": 0.5,
    "PiStacking_T": 0.5,
    "SaltBridge_lneg": 0.5,
    "SaltBridge_pneg": 0.5,
    "PiCation_lring": 0.5,
    "PiCation_pring": 0.5,
    "XBond": 0.5,
    "HBond_ldon": 0.5,
    "HBond_pdon": 0.5,
    "Hydrophobic": 0.5,
}

In [5]:
module = ModelingModule('model_weights/model.tar', 'cuda',
                        score_threshold=score_threshold)

In [6]:
failed = []
for rec_id, lig_id in tqdm(split_file.items()):
    try:
        rec_path = os.path.join('dataset/crossdock', rec_id + '_rec.pdb')
        lig_path = os.path.join('dataset/crossdocked_pocket10', lig_id)
        out_path = os.path.join('dataset/new_pharmacophores', rec_id + '.pt')
        
        p_model = module.run(rec_path, ref_ligand_path=lig_path)
        p_model.save(out_path)
    except:
        failed.append((rec_id, lig_id))

 14%|█▎        | 2069/15307 [49:54<6:53:13,  1.87s/it]==============================
*** Open Babel Warning  in ReadMolecule
There may be erroneous addition spaces or
the file may contains Atom Lists, which are ignored

 64%|██████▍   | 9863/15307 [4:16:10<2:17:49,  1.52s/it]